In [17]:
#importing all the necessary libraries
import math
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import requests
import json
from datetime import datetime, timedelta
import gzip
import os
import time

In [ ]:
#extracting the data from Polygon.io using the API
def fetch_forex_data_from_polygon(api_key, date):
    
    # Polygon.io forex endpoint for grouped daily data
    url = f"https://api.polygon.io/v2/aggs/grouped/locale/global/market/fx/{date}"
    
    params = {
        'adjusted': 'true',
        'apikey': api_key
    }
    
    print(f"Fetching forex data for {date}...")
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
        return None, response
    
    data = response.json()
    
    if 'results' not in data:
        print("No results found in the response")
        return None, response
    
    # Convert to DataFrame format similar to your existing structure
    records = []
    for result in data['results']:
        ticker = result['T']  # Ticker symbol like C:EURUSD
        
        # Skip if not a forex pair or doesn't match currency filter
        if not ticker.startswith('C:'):
            continue
            
        # if currencies:
        # Extract currency pair from ticker (e.g., C:EURUSD -> EUR, USD)
        pair = ticker[2:]  # Remove 'C:' prefix
        if len(pair) == 6:  # Standard format like EURUSD
            currency1, currency2 = pair[:3], pair[3:]
        else:
            # Handle other formats like EUR-USD
            parts = pair.replace('-', '').replace('/', '')
            if len(parts) == 6:
                currency1, currency2 = parts[:3], parts[3:]
            else:
                continue
                    
            # if currency1 not in currencies or currency2 not in currencies:
            #     continue
        
        # Create record with similar structure to your existing data
        record = {
            'ticker': ticker,
            'participant_timestamp': int(result['t']) * 1000000,  # Convert to nanoseconds
            # Add other fields if available
            'ask_price': result.get('a', 0),
            'bid_price': result.get('b', 0),
            'open': result.get('o', 0),
            'high': result.get('h', 0),
            'low': result.get('l', 0),
            'close': result.get('c', 0),
            'volume': result.get('v', 0)
        }
        records.append(record)
    
    df = pd.DataFrame(records)
    
    
    print(f"Fetched {len(df)} forex records")
    return df, response

POLYGON_API_KEY = "vSmtvpdrv8xGtZG7YYgSiPg3irGrpWbd"  

In [24]:
#extracting the data of each day for 3 months for targetted currencies
# taking the top most traded currencies and INR
df = pd.DataFrame()
date_range = pd.date_range(start="2025-01-01", end="2025-03-30", freq='D')

timedelta_minutes = timedelta(minutes=1)
for date in date_range:

    date_str = date.strftime("%Y-%m-%d") 
    daily_df, response = fetch_forex_data_from_polygon(
        api_key=POLYGON_API_KEY,
        date=date_str,  # YYYY-MM-DD format
    )
    if daily_df is None:
        #if error type is 429, wait for 60 seconds
        if response.status_code == 429:
            print("Rate limit exceeded. Waiting for 60 seconds...")
            time.sleep(60)
        
    if daily_df is not None:
        df = pd.concat([df, daily_df], ignore_index=True) 

Fetching forex data for 2025-01-01...
Fetched 670 forex records
Fetching forex data for 2025-01-02...
Fetched 1175 forex records
Fetching forex data for 2025-01-03...
Fetched 1171 forex records
Fetching forex data for 2025-01-04...
No results found in the response
Fetching forex data for 2025-01-05...
Fetched 1171 forex records
Fetching forex data for 2025-01-06...
Error: 429 - {"status":"ERROR","request_id":"cbdba5004b6014f0e42dbf53c3f900be","error":"You've exceeded the maximum requests per minute, please wait or upgrade your subscription to continue. https://polygon.io/pricing"}
Rate limit exceeded. Waiting for 60 seconds...
Fetching forex data for 2025-01-07...
Fetched 1220 forex records
Fetching forex data for 2025-01-08...
Fetched 1211 forex records
Fetching forex data for 2025-01-09...
Fetched 1221 forex records
Fetching forex data for 2025-01-10...
Fetched 1231 forex records
Fetching forex data for 2025-01-11...
No results found in the response
Fetching forex data for 2025-01-12

In [25]:


# Save the combined DataFrame to a gzipped CSV
df.to_csv("data_set.csv.gz", compression='gzip', index=False)
df

,ticker,participant_timestamp,open,high,low,close,volume
0,C:AUDCAD,1735775999999000000,0.890400,0.891829,0.888030,0.890250,5708
1,C:CADJPY,1735775999999000000,109.290000,109.747000,108.620000,109.704000,5859
2,C:USDEGP,1735775999999000000,50.818484,50.825360,50.748400,50.801600,392
3,C:BRLARS,1735775999999000000,166.691200,166.878230,163.144698,163.550800,580
4,C:CNHUSD,1735775999999000000,0.136323,0.136396,0.136110,0.136362,3176
...,...,...,...,...,...,...,...
79836,C:CADVND,1743379199999000000,17848.161395,17848.161395,17848.161395,17848.161395,1
79837,C:NZDINR,1743379199999000000,48.834077,48.834077,48.834077,48.834077,1
79838,C:JPYZAR,1743379199999000000,0.122656,0.123411,0.122555,0.123109,4042
79839,C:DKKSEK,1743379199999000000,1.444860,1.452780,1.444860,1.451770,6161
